# Data Analysis 3
---
Data sets:<br>
-[US Minimum Wage by State from 1968 to 2020](https://www.kaggle.com/lislejoem/us-minimum-wage-by-state-from-1968-to-2017)<br>
-[US Unemployment Rate by County, 1990-2016](https://www.kaggle.com/jayrav13/unemployment-by-county-us)<br>
-[2016 US Presidential Election Vote By County](https://www.kaggle.com/stevepalley/2016uspresidentialvotebycounty)<br>
Main ideas from [pythonprogramming.net](https://pythonprogramming.net/combining-datasets-python3-pandas-data-analysis/)

### Abstract
Merging 3 datasets to see the correlation and covariance of minimum wage, unemployment rate and percentage of votes to a candidate. Note that this is just to practice merging datasets, we left many data aside and the results are in no way significant. 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from seaborn import set_theme
set_theme()

In [6]:
unemp_country = pd.read_csv("./archive/output.csv")
unemp_country.head()

,Year,Month,State,County,Rate
0,2015,February,Mississippi,Newton County,6.1
1,2015,February,Mississippi,Panola County,9.4
2,2015,February,Mississippi,Monroe County,7.9
3,2015,February,Mississippi,Hinds County,6.1
4,2015,February,Mississippi,Kemper County,10.6


Tomando lo del análisis pasado

In [8]:
df = pd.read_csv("./archive/minwage.csv")
min_wage = pd.DataFrame()

for name, group in df.groupby("State"):
    if min_wage.empty:
        min_wage = group.set_index("Year")[["Department.Of.Labor.Cleaned.Low.Value.2020.Dollars"]].rename(columns={'Department.Of.Labor.Cleaned.Low.Value.2020.Dollars': name})
    else:
        min_wage = min_wage.join( group.set_index("Year")[["Department.Of.Labor.Cleaned.Low.Value.2020.Dollars"]].rename(columns={'Department.Of.Labor.Cleaned.Low.Value.2020.Dollars': name}))

min_wage = min_wage.replace(0, np.nan).dropna(axis=1)
min_wage.head()

,Alaska,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,Guam,Hawaii,Idaho,...,Oregon,Pennsylvania,Rhode Island,South Dakota,Utah,Vermont,Washington,West Virginia,Wisconsin,Wyoming
Year,,,,,,,,,,,,,,,,,,,,,
1968,15.61,1.16,12.26,7.43,10.41,9.29,9.29,9.29,9.29,8.55,...,9.29,8.55,10.41,3.16,7.43,10.41,11.89,7.43,9.29,8.92
1969,14.80,1.10,11.63,7.05,9.87,8.81,8.81,8.81,8.81,8.11,...,8.81,8.11,9.87,3.00,7.05,9.87,11.28,7.05,8.81,8.46
1970,14.00,7.33,11.00,6.67,10.67,8.33,10.67,10.67,10.67,8.33,...,8.33,8.67,10.67,6.67,6.67,10.67,10.67,6.67,8.67,8.67
1971,13.41,7.03,10.54,6.39,10.22,7.98,10.22,10.22,10.22,7.98,...,7.98,8.30,10.22,6.39,6.39,10.22,10.22,6.39,8.30,8.30
1972,12.99,7.43,10.21,6.19,11.45,9.90,9.90,11.76,9.90,8.66,...,7.74,9.90,9.90,6.19,7.43,9.90,9.90,7.43,8.97,9.28


Veremos como se relacionan el salario mínimo y el desempleo por estado. Para esto sería ideal poder unir nuestro dataset anterior al nuevo, pero al tener tamaños distintos en el número de condados obtendríamos un sliding error.

Con fines de continuar las ideas principales utilizaremos estos datasets con los valores que ya tenemos, pero lo ideal sería conseguir las funciones faltantes

In [10]:
def get_min_wage(year, state):
    try:
        return min_wage.loc[year][state]
    except:
        return np.nan

La forma en que lo uniremos funciona siempre, sin embargo, no es la más rápida ni eficiente, pero si es muy poderosa. Haremos un mapa de la función con las variables y añadiremos el resultado a una nueva columna en nuestro dataframe

In [11]:
%%time

unemp_country['min_wage'] = tuple(map(get_min_wage, unemp_country['Year'], unemp_country['State']))

Wall time: 53.8 s


In [31]:
unemp_country.tail()

,Year,Month,State,County,Rate,min_wage
885543,2009,November,Maine,Somerset County,10.5,8.74
885544,2009,November,Maine,Oxford County,10.5,8.74
885545,2009,November,Maine,Knox County,7.5,8.74
885546,2009,November,Maine,Piscataquis County,11.3,8.74
885547,2009,November,Maine,Aroostook County,9.0,8.74


In [13]:
unemp_country[["Rate", "min_wage"]].corr()

,Rate,min_wage
Rate,1.000000,0.153035
min_wage,0.153035,1.000000


In [14]:
unemp_country[["Rate", "min_wage"]].cov()

,Rate,min_wage
Rate,9.687873,0.673099
min_wage,0.673099,2.000364


In [18]:
pres16 = pd.read_csv("./archive/pres16results.csv")
pres16

,county,fips,cand,st,pct_report,votes,total_votes,pct,lead
0,NaN,US,Donald Trump,US,0.9951,60350241.0,127592176.0,0.472993,Donald Trump
1,NaN,US,Hillary Clinton,US,0.9951,60981118.0,127592176.0,0.477938,Donald Trump
2,NaN,US,Gary Johnson,US,0.9951,4164589.0,127592176.0,0.032640,Donald Trump
3,NaN,US,Jill Stein,US,0.9951,1255968.0,127592176.0,0.009844,Donald Trump
4,NaN,US,Evan McMullin,US,0.9951,451636.0,127592176.0,0.003540,Donald Trump
...,...,...,...,...,...,...,...,...,...
18470,Yauco Municipio,72153,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18471,Midway Islands,74300,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18472,St. Croix Island,78010,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18473,St. John Island,78020,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
county_2016 = unemp_country.copy()[(unemp_country['Year']==2016) & (unemp_country['Month']=='November')]
county_2016.tail()

,Year,Month,State,County,Rate,min_wage
102225,2016,November,Maine,Somerset County,5.4,8.08
102226,2016,November,Maine,Oxford County,4.6,8.08
102227,2016,November,Maine,Knox County,3.6,8.08
102228,2016,November,Maine,Piscataquis County,5.6,8.08
102229,2016,November,Maine,Aroostook County,5.7,8.08


In [43]:
state_abbv = pd.read_csv("./archive/state_abbv.csv", index_col = 0)
state_abbv = state_abbv[["Postal Code"]]
state_abbv.head()

,Postal Code
State/District,
Alabama,AL
Alaska,AK
Arizona,AZ
Arkansas,AR
California,CA


In [44]:
state_abbv_dict = state_abbv.to_dict()['Postal Code']
state_abbv_dict

{'Alabama': 'AL',
 'Alaska': 'AK',
 'Arizona': 'AZ',
 'Arkansas': 'AR',
 'California': 'CA',
 'Colorado': 'CO',
 'Connecticut': 'CT',
 'Delaware': 'DE',
 'District of Columbia': 'DC',
 'Florida': 'FL',
 'Georgia': 'GA',
 'Hawaii': 'HI',
 'Idaho': 'ID',
 'Illinois': 'IL',
 'Indiana': 'IN',
 'Iowa': 'IA',
 'Kansas': 'KS',
 'Kentucky': 'KY',
 'Louisiana': 'LA',
 'Maine': 'ME',
 'Maryland': 'MD',
 'Massachusetts': 'MA',
 'Michigan': 'MI',
 'Minnesota': 'MN',
 'Mississippi': 'MS',
 'Missouri': 'MO',
 'Montana': 'MT',
 'Nebraska': 'NE',
 'Nevada': 'NV',
 'New Hampshire': 'NH',
 'New Jersey': 'NJ',
 'New Mexico': 'NM',
 'New York': 'NY',
 'North Carolina': 'NC',
 'North Dakota': 'ND',
 'Ohio': 'OH',
 'Oklahoma': 'OK',
 'Oregon': 'OR',
 'Pennsylvania': 'PA',
 'Rhode Island': 'RI',
 'South Carolina': 'SC',
 'South Dakota': 'SD',
 'Tennessee': 'TN',
 'Texas': 'TX',
 'Utah': 'UT',
 'Vermont': 'VT',
 'Virginia': 'VA',
 'Washington': 'WA',
 'West Virginia': 'WV',
 'Wisconsin': 'WI',
 'Wyoming': 'WY

In [46]:
county_2016['State'] = county_2016['State'].map(state_abbv_dict)
county_2016.tail()

,Year,Month,State,County,Rate,min_wage
102225,2016,November,ME,Somerset County,5.4,8.08
102226,2016,November,ME,Oxford County,4.6,8.08
102227,2016,November,ME,Knox County,3.6,8.08
102228,2016,November,ME,Piscataquis County,5.6,8.08
102229,2016,November,ME,Aroostook County,5.7,8.08


In [47]:
print(len(county_2016))
print(len(pres16))

2821
18475


In [48]:
pres16.rename(columns={"county": "County", "st": "State"}, inplace=True)
pres16.head()

,County,fips,cand,State,pct_report,votes,total_votes,pct,lead
0,NaN,US,Donald Trump,US,0.9951,60350241.0,127592176.0,0.472993,Donald Trump
1,NaN,US,Hillary Clinton,US,0.9951,60981118.0,127592176.0,0.477938,Donald Trump
2,NaN,US,Gary Johnson,US,0.9951,4164589.0,127592176.0,0.032640,Donald Trump
3,NaN,US,Jill Stein,US,0.9951,1255968.0,127592176.0,0.009844,Donald Trump
4,NaN,US,Evan McMullin,US,0.9951,451636.0,127592176.0,0.003540,Donald Trump


In [49]:
for df in [county_2016, pres16]:
    df.set_index(["County", "State"], inplace=True)

In [50]:
donald = pres16.copy()[pres16['cand'] == 'Donald Trump']
donald = donald[['pct']]
donald.dropna(inplace=True)
donald.head()

pct
County State          
NaN    US     0.472993
       CA     0.330641
       FL     0.490640
       TX     0.525830
       NY     0.374752

In [53]:
all_together = county_2016.merge(donald, on=["County", "State"])
all_together.dropna(inplace=True)
all_together.drop("Year", axis=1, inplace=True)
all_together.head()

,,Month,Rate,min_wage,pct
County,State,,,,
Major County,OK,November,3.9,2.16,0.864960
Pottawatomie County,OK,November,4.4,2.16,0.701342
Johnston County,OK,November,5.7,2.16,0.770057
Jefferson County,OK,November,7.1,2.16,0.812367
Beaver County,OK,November,2.7,2.16,0.888243


In [54]:
all_together.corr()

,Rate,min_wage,pct
Rate,1.000000,-0.014471,-0.023618
min_wage,-0.014471,1.000000,-0.302875
pct,-0.023618,-0.302875,1.000000


In [55]:
all_together.cov()

,Rate,min_wage,pct
Rate,2.917350,-0.041893,-0.006125
min_wage,-0.041893,2.872549,-0.077945
pct,-0.006125,-0.077945,0.023056
